# Kaggle Featured Prediction Competition: H&M Personalized Fashion Recommendations

In this [competition](https://www.kaggle.com/competitions/h-and-m-personalized-fashion-recommendations), product recommendations have to be done based on previous purchases. There's a whole range of data available including customer meta data, product meta data, and meta data that spans from simple data, such as garment type and customer age, to text data from product descriptions, to image data from garment images.

In this notebook we will be working with implicit's ALS library for our recommender systems. Please do check out the [docs](https://benfred.github.io/implicit/index.html) for more information.

## Install necessary packages

We can install the necessary package by either running `pip install --user <package_name>` or include everything in a `requirements.txt` file and run `pip install --user -r requirements.txt`. We have put the dependencies in a `requirements.txt` file so we will use the former method.

Restart the kernel after installation

In [2]:
!pip install --user -r requirements.txt

     |████████████████████████████████| 18.6 MB 8.6 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 58 kB 8.3 MB/s             
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 78 kB 7.2 MB/s             
     |████████████████████████████████| 78 kB 10.0 MB/s            
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=2381 sha256=b097d9ed6edd1c879d42180dffc20aa2659b2628f9711ae4204ce4b1cb636dc2
  Stored in directory: /home/jovyan/.cache/pip/wheels/23/9d/42/5ec745cbbb17517000a53cecc49d6a865450d1f5cb16dc8a9c
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73994 sha256=6e228faf71a3c29b864d64cfe8fe5b59a1257cfc4afb24f7c4712943489a5710
  Stored in directory: /home/jovyan/.cache/pip/wheels/77/47/e4/44a4ba1b7dfd53faaa35f59f1175e123b213ff401a8a56876b
Successfully built sklearn kaggle


## Download Data from Kaggle

Download relevant data from kaggle by running the below code cell. Follow the initial steps information mentioned in Github README.md to get the Kaggle username and key for authentication of Kaggle Public API. There's no need of secret to be created for the following step. The credentials will be present in the kaggle.json file. This cell needs to be run before starting Kale pipeline from  Kale deployment panel. Please ensure that you run the cell only once so you don't create nested directories. Restart the kernel before running the code cell again. 

In [1]:
import os


# Get the Kaggle Username and password from the kaggle.json file
# and paste it in place of KAGGLE_USERNAME AND KAGGLE_KEY on right hand side

os.environ['KAGGLE_USERNAME'] = "KAGGLE_USERNAME"
os.environ['KAGGLE_KEY'] = "KAGGLE_KEY"

path = "data/"

os.chdir(os.getcwd())
os.system("mkdir " + path)
os.chdir(path)

import kaggle
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()

# Getting the files list from Kaggle using Kaggle api
file_list = api.competition_list_files('h-and-m-personalized-fashion-recommendations')

# Download the required files individually. You can also choose to download the entire dataset if you want to work with image data as well. The files will be in downloaded   
api.competition_download_file('h-and-m-personalized-fashion-recommendations','customers.csv')
api.competition_download_file('h-and-m-personalized-fashion-recommendations','transactions_train.csv')
api.competition_download_file('h-and-m-personalized-fashion-recommendations','articles.csv')
api.competition_download_file('h-and-m-personalized-fashion-recommendations','sample_submission.csv')   

print("Unzipping the files ...")

# Get the path of the directory where the files are downloaded
path_dir = os.getcwd()

from zipfile import ZipFile 

# Extracting all files from individual zip files
zipfile1 = ZipFile(path_dir + '/customers.csv.zip', 'r')
zipfile1.extract("customers.csv")
zipfile1.close()
    
zipfile2 = ZipFile(path_dir + '/transactions_train.csv.zip', 'r')
zipfile2.extract("transactions_train.csv")
zipfile2.close()
    
zipfile3 = ZipFile(path_dir + '/articles.csv.zip', 'r')
zipfile3.extract("articles.csv")
zipfile3.close()
    
zipfile4 = ZipFile(path_dir + '/sample_submission.csv.zip', 'r')
zipfile4.extract("sample_submission.csv")
zipfile4.close()

print("Checking the files are extracted properly ...")

for file in os.listdir(path_dir):
     filename = os.fsdecode(file)
     if filename.endswith(".csv"):
        file_size = os.path.getsize(path_dir + "/" + filename)
        if file_size< 1e9:
            file_size = str(round(file_size/(1024*1024))) + "MB"
        else:
            file_size = str(round(file_size/(1024*1024*1024))) + "GB"
        for file in file_list:
            if file.name == filename and file.size == file_size:
                print(file.name,file.size, file_size)

print("All files are downloaded and unzipped inside the data directory. Please move on to next step")


customers.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
transactions_train.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
articles.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
sample_submission.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
Unzipping the files ...


Checking the files are extracted properly ...


sample_submission.csv 258MB 258MB
customers.csv 198MB 198MB
transactions_train.csv 3GB 3GB
articles.csv 34MB 34MB
All files are downloaded and unzipped inside the data directory. Please move on to next step


## Imports

In [ ]:
import numpy as np
import pandas as pd
import implicit 
import scipy.sparse as sparse

## Load Data

In [ ]:
path = "data/"
train_data_filepath = path + "transactions_train.csv"
article_metadata_filepath = path + "articles.csv"
customer_metadata_filepath = path + "customers.csv"
test_data_filepath = path + "sample_submission.csv"

In [ ]:
train_data = pd.read_csv(train_data_filepath)
test_data = pd.read_csv(test_data_filepath)
customer_data = pd.read_csv(customer_metadata_filepath)
article_data = pd.read_csv(article_metadata_filepath)

## Exploring the dataset

In [ ]:
train_data.head()

In [ ]:
train_data.info()

In [ ]:
train_data.describe()

In [ ]:
test_data.tail()

In [ ]:
customer_data.tail()

In [ ]:
article_data.tail()

In [ ]:
# We will be dropping t_dat, sales_channel and price as this won't be part of the recommendation system we will be building 
train_data.drop(['t_dat','sales_channel_id','price'], axis= 1, inplace = True)

In [ ]:
train_data.head()

## Preprocess Data

In [ ]:
# create a new purchase count column that would gives us count of every article bought by the customers
X = train_data.groupby(['customer_id', 'article_id'])['article_id'].count().reset_index(name = "purchase_count") 

# Getting unique number of customers and articles using the customer and article metadata data files
unique_customers = customer_data['customer_id'].unique()
unique_articles = article_data['article_id'].unique()

# length of the customers and articles
n_customers = len(unique_customers)
n_articles = len(unique_articles)

# Create a mapping for customer_id to convert it from an object column to an int column for the sparse matrix creation
customer_id_dict = {unique_customers[i]:i  for i in range(len(unique_customers))}
reverse_customer_id_dict = {i:unique_customers[i] for i in range(len(unique_customers))} 
numeric_cus_id = []
for i in range(len(X['customer_id'])):
    numeric_cus_id.append(customer_id_dict.get(X['customer_id'][i]))
X['customer_id'] = numeric_cus_id

# Create a mapping for article_id so that the sparse matrix creation doesn't get large enough due to long int values of article_ids
article_id_dict = {unique_articles[i]:i  for i in range(len(unique_articles))}
reverse_article_id_dict = {i:unique_articles[i] for i in range(len(unique_articles))}
numeric_art_id = []
for i in range(len(X['article_id'])):
    numeric_art_id.append(article_id_dict.get(X['article_id'][i]))
X['article_id'] = numeric_art_id

In [ ]:
X.head()

## Sparse Matrix Creation

In [ ]:
# Constructing sparse matrices for alternating least squares algorithm    
sparse_user_item_coo = sparse.coo_matrix((X.purchase_count, (X.customer_id, X.article_id)), shape = (n_customers, n_articles))
sparse_user_item_csr = sparse.csr_matrix((X['purchase_count'], (X['customer_id'], X['article_id'])), shape = (n_customers, n_articles))

In [ ]:
sparse_user_item_csr

## Model Training

In [ ]:
# parameters for the model
als_params = dict(
    factors = 200,         # number of latent factors - try between 50 to 1000
    regularization = 0.01, # regularization factor - try between 0.001 to 0.2
    iterations = 5,        # iterations            - try between 2 to 100
)

# initialize a model
model = implicit.als.AlternatingLeastSquares(**als_params)

# train the model on a sparse matrix of user/item/confidence weights    
model.fit(sparse_user_item_csr)

In [ ]:
model

## Predictions

In [ ]:
predictions=[]
count = 0
for cust_id in test_data.customer_id:
    cust_id = customer_id_dict.get(cust_id)
    if(cust_id!=None):    
        recommendations = model.recommend(cust_id, sparse_user_item_csr[cust_id],10)
        result=[]
        for i in range(len(recommendations[0])):
            val = reverse_article_id_dict.get(recommendations[0][i])
            result.append(val)  
        predictions.append(result)


In [ ]:
test_data['prediction'] = predictions
test_data

### Final Submission

In [ ]:
test_data.to_csv('data/submission.csv', index=False)